## <font color="green">Writing a video using OpenCV</font>

While building applications, it becomes important to save demo videos of your work as well as many applications themselves might require saving a video clip. For example, in a surveiallance application, you might have to save a video clip as soon as you see something unusual happening.

In this notebook, we will describe how to save a video in **avi** and **mp4** formats using openCV.


In [2]:
# import the library
import os
import cv2
import matplotlib.pyplot as plt
import numpy

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import YouTubeVideo, display, HTML
from base64 import b64encode

%matplotlib inline


import os


def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assests....", end="")

    # Downloading zip file using urllib package.
    urlretrieve(url, save_path)

    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(save_path) as z:
            # Extract ZIP file contents in the same directory.
            z.extractall(os.path.split(save_path)[0])

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)


URL = r"https://www.dropbox.com/s/p8h7ckeo2dn1jtz/opencv_bootcamp_assets_NB6.zip?dl=1"

asset_zip_path = os.path.join(os.getcwd(), "opencv_bootcamp_assets_NB6.zip")

# Download if assest ZIP does not exists.
if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)

source = 'race_car.mp4' # source = 0 for webcam

cap = cv2.VideoCapture(source)

if not cap.isOpened():
    print("Error opening video stream or file")

ret, frame = cap.read()  # Read and display one frame
plt.imshow(frame[..., ::-1])

video = YouTubeVideo("RwxVEjv78LQ", width=700, height=438)  #Display the video file
#display(video)

## Read Video from Source

# Default resolutions of the frame are obtained.
# Convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.
out_avi = cv2.VideoWriter(
    "race_car_out.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 10, (frame_width, frame_height)
)


out_mp4 = cv2.VideoWriter(
    "race_car_out.mp4", cv2.VideoWriter_fourcc(*"XVID"), 10, (frame_width, frame_height)
)

#Read frames and write to file. Read until video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break
    out_avi.write(frame)
    out_mp4.write(frame)

# When everything done, release the VideoCapture and VideoWriter objects
cap.release()
out_avi.release()
out_mp4.release()

# Installing ffmpeg

!apt-get -qq install ffmpeg


# Change video encoding of mp4 file from XVID to h264
!ffmpeg -y -i "/content/race_car_out.mp4" -c:v libx264 "race_car_out_x264.mp4"  -hide_banner -loglevel error

mp4 = open("/content/race_car_out_x264.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""<video width=700 controls><source src="{data_url}" type="video/mp4"></video>""")

video = YouTubeVideo("2Gju7YLfkP0", width=700, height=438)
display(video)

ModuleNotFoundError: No module named 'cv2'

# <font color="green">Using Camera filters in image processing</font>


OpenCV provides various image processing techniques that can be used as camera filters to modify the appearance of images or videos captured by a camera in real-time. Some common camera filters and their corresponding OpenCV functions include:

1. **Blur Filter**: Blurring is a common image processing technique used to reduce noise or smooth out details in an image. OpenCV provides functions like `cv2.GaussianBlur()` for applying Gaussian blur, `cv2.medianBlur()` for median blur, and `cv2.bilateralFilter()` for bilateral filtering.

Bilateral filtering is an advanced method of image smoothing that, while reducing noise, maintains the edges in an image²⁴. This is in contrast to techniques like Gaussian blurring, which blur out the edges as well¹. The main property of Bilateral Filtering is that it does not do averaging across edges. That is why it is also called an edge-preserving filter². 

However, it's important to note that Bilateral Filter is not faster than other blurring filters in OpenCV⁵. It is a non-linear, edge-preserving, and noise-reducing smoothing filter². But it does not only blur the edges in an image¹², and it is not limited to grayscale images³. It can be applied to color images as well³.

Source: Conversation with Bing, 5/3/2024
(1) Bilateral Filtering in Python OpenCV with cv2.bilateralFilter() - MLK. https://machinelearningknowledge.ai/bilateral-filtering-in-python-opencv-with-cv2-bilateralfilter/.
(2) Bilateral Filtering in OpenCV. https://www.devanddep.com/tutorial/opencv/python-bilateral-filtering.html.
(3) OpenCV: Smoothing Images. https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html.
(4) Bilateral Smoothing - OpenCV Tutorial C++. https://www.opencv-srf.com/2018/03/bilateral-smoothing.html.
(5) Bilateral Filtering of the Image using OpenCV | Lindevs. https://lindevs.com/bilateral-filtering-of-the-image-using-opencv/.
(6) en.wikipedia.org. https://en.wikipedia.org/wiki/OpenCV.

2. **Edge Detection**: Edge detection algorithms identify the boundaries of objects within an image. OpenCV offers several edge detection methods, such as the Canny edge detector (`cv2.Canny()`), Sobel operator (`cv2.Sobel()`), and Laplacian operator (`cv2.Laplacian()`).

3. **Color Adjustment**: OpenCV allows you to adjust the color properties of an image, such as brightness, contrast, saturation, and hue. Functions like `cv2.convertScaleAbs()` can be used for simple brightness and contrast adjustments, while `cv2.cvtColor()` can be used to convert between different color spaces.

4. **Thresholding**: Thresholding is a technique used to create binary images by segmenting regions of interest based on pixel intensity values. OpenCV provides functions like `cv2.threshold()` for simple thresholding, `cv2.adaptiveThreshold()` for adaptive thresholding, and `cv2.inRange()` for range-based thresholding.

5. **Filter Effects**: OpenCV offers functions to apply various filter effects to images, such as sepia (`cv2.sepiaFilter()`), pencil sketch (`cv2.pencilSketch()`), and cartoon (`cv2.stylization()`).

These are just a few examples of camera filters that can be implemented using OpenCV. Depending on your application requirements, you can combine these techniques or develop custom filters to achieve the desired visual effects in real-time camera applications.


Next, some of the more common image processing techniques are demonstrated, ones which are often used in computer vision pipelines. To do this, we’re going to build on the camera demonstration from the last video where we sent streaming video from the camera to an output window in the display. However, this time, we’ll be doing some image processing on the video frames first and then send those results to the output display. Below, we’re defining the four different run modes for the script which include a preview mode, a blurring filter, a corner feature detector, and a canny edge detector.


In [ ]:
PREVIEW = 0
  # Preview Mode
BLUR = 1
  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY = 3
  # Canny Edge Detector

The constants for different image processing modes in the code. Here's a brief explanation of each mode:

1. **PREVIEW (Mode 0)**: This mode is for previewing the original image without any additional processing. It likely displays the raw image captured by the camera or loaded from a file.

2. **BLUR (Mode 1)**: This mode applies a blurring filter to the image. Blurring is a common image processing technique used to reduce noise or smooth out details in an image. There are various types of blurring filters available, such as Gaussian blur, median blur, and bilateral blur.

3. **FEATURES (Mode 2)**: This mode is for detecting corner features in the image. Corner detection is a fundamental technique in computer vision used to identify important points or keypoints in an image. These keypoints often represent distinctive features that can be used for tasks like image matching, object recognition, and camera calibration.

4. **CANNY (Mode 3)**: This mode applies the Canny edge detector to the image. The Canny edge detector is a popular algorithm used to detect edges in images. It works by finding the gradients of the image intensity and then identifying the local maxima of these gradients as potential edge points. Canny edge detection is commonly used as a preprocessing step for various computer vision tasks, including object detection and image segmentation.

These modes provide different functionalities for processing images and extracting useful information from them, depending on the requirements of your application.


In [ ]:
feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

we’re defining a small dictionary of parameter settings for the corner feature detector.

These include the maximum number of corners that the algorithm will return.

The quality level is a parameter for characterizing the minimum acceptable quality of image corners. The way that works is that the corner feature with the highest value in the entire image is multiplied by this parameter and then that value is used as a minimum threshold for filtering corner features from the final list that’s returned by the algorithm.

The next parameter, the minimum distance, is the minimum distance between adjacent feature corners and this is measured in pixel space. It’s the Euclidean distance in pixel space which describes how close two corner features can be in the list that’s returned from the algorithm.

Finally, block size is the size of the pixel neighborhood that is used in the algorithm for computing the feature corners.


In [ ]:
import sys

s = 0
if len(sys.argv) > 1:
    s = sys.argv[1]

image_filter = PREVIEW
alive = True

win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(s)

Above code snippet is very similar to the code in the previous task where we’re setting the device index for the camera, creating an output window for the streamed results, and then creating a video capture object so that we can process the video stream in the loop below.


In [ ]:
import numpy

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        result = cv2.Canny(frame, 80, 150)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))
    elif image_filter == FEATURES:
        result = frame
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        if corners is not None:
            for x, y in numpy.float32(corners).reshape(-1, 2):
                cv2.circle(result, (x, y), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("C") or key == ord("c"):
        pass
    elif key == ord("B") or key == ord("b"):
        pass
    elif key == ord("F") or key == ord("f"):
        pass
    elif key == ord("P") or key == ord("p"):
        pass

we enter a while loop and the first line in that loop is to read a frame from the video stream. Then, I’m going to flip that frame horizontally mainly as a convenience for myself so that it’s easier for me to point things out in the video stream.

Then depending on the run configuration for the script, we’ll be executing one of these functions in OpenCV. Of course, if we’re in preview mode, we’re simply going to take the frame and set that to the result and then display that directly to the output window using imshow.

But for these other run modes, we’re going to do some processing first and then send the processed results to the output window. Starting on line 71, we’re calling the canny edge detection function in OpenCV. The first argument there is the image frame and then there’s two additional arguments, a lower threshold, and an upper threshold.

The upper threshold is used for deciding whether or not a series of pixels should be considered as an edge. So if the intensity gradient of those pixels exceeds the upper threshold, then we’ll declare those pixels as constituting a sure edge. Likewise, for pixels whose intensity gradients are below the lower threshold, those segments will be completely discarded.

However, for the pixels whose gradients fall in between these two thresholds, we’ll consider those as candidate edges if they can be associated with a nearby segment that has already been declared as an edge. In other words, we’re allowing for weaker edges to be connected to stronger ones if the weaker edges are likely to be along the same true edge.

Then the next function here is a blur function in OpenCV. This blur function uses a box filter to blur the image. The first input to this function is the image frame itself and then this second input are the dimensions for the box kernel. This would be a 13 by 13 box kernel that would be convolved with the image to result in a blurred image. If the size of the kernel is smaller, then the blurring is less. If the size of the kernel is larger, then you get a more substantial blurring.

Finally, for the corner feature detector, we’re converting the frame, the image frame, to a grayscale image. Then on line 77, we’re going to call the function good features to track. Although it isn’t indicated in the name of this function, what this function does is compute corner features. The first argument is a grayscale image of the video frame and then the second argument is a dictionary of feature parameters that we described up above.

What that returns is a list of corners that were detected in the image. If we have one or more corners detected, then we’re going to simply annotate the result with small green circles to indicate the locations of those features.


### <font color="green">goodFeaturesToTrack() function </font>


In OpenCV (Open Source Computer Vision Library), `goodFeaturesToTrack()` is a function used to detect stable corners (interest points) in an image that are suitable for tracking in subsequent frames of a video or image sequence. These corners are often referred to as "good features to track" because they are expected to be:

- **Distinctive:** They should have a clear and well-defined structure in the image, allowing them to be easily distinguished from their surrounding pixels.
- **Stable:** They should remain relatively unchanged in appearance across different frames of the video sequence, even under slight variations in lighting or perspective.

Here's a breakdown of the function's syntax and usage:

**Syntax:**

```python
corners = cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance, blockSize=3, useHarrisDetector=False, k=0.04)
```

**Parameters:**

- **image:** This is the grayscale image on which you want to detect corners.
- **maxCorners:** This is the maximum number of corners to be detected. The function will identify up to this number of corners with the strongest response.
- **qualityLevel:** This is a parameter between 0.0 and 1.0 that specifies the minimum quality of a corner to be retained. Corners with a higher response value (closer to 1.0) are considered more stable and reliable for tracking.
- **minDistance:** This is the minimum distance (in pixels) between the detected corners. This helps avoid detecting corners that are very close together, which can be redundant for tracking purposes.
- **blockSize (optional):** This specifies the size of the neighborhood area used to determine the corner response at each pixel. Defaults to 3.
- **useHarrisDetector (optional):** This is a boolean flag. If set to True, the Harris corner detector is used. Otherwise, the Shi-Tomasi corner detector is used (default). Both methods identify corners based on local variations in intensity, but they have slightly different characteristics.
- **k (optional):** This is a parameter used in the Shi-Tomasi corner detector for thresholding corner responses. Defaults to 0.04.

**Return Value:**

- The function returns a NumPy array containing the detected corner coordinates (x, y) for each corner.

**Applications:**

`goodFeaturesToTrack()` is a fundamental function in computer vision tasks like:

- **Feature Tracking:** It's often used as the first step in feature tracking algorithms, where the detected corners are tracked across multiple video frames to estimate motion or object movement.
- **Structure from Motion (SfM):** This technique for reconstructing 3D scenes from multiple images relies on identifying and tracking corresponding corners across different viewpoints.
- **Object Recognition:** Corner features can be used as descriptors for object recognition algorithms that identify objects based on their distinctive corners and edges.

**Using `goodFeaturesToTrack()` Effectively:**

- **Image Preprocessing:** Grayscale conversion is often recommended before applying `goodFeaturesToTrack()`, as it simplifies the corner detection process.
- **Parameter Tuning:** The choice of parameters like `maxCorners`, `qualityLevel`, and `minDistance` can significantly impact the number and quality of detected corners. Experiment with these values to achieve the desired results for your specific application.

Here's an example of using `goodFeaturesToTrack()` to detect corners in an image:

```python
import cv2
import numpy as np

# Load an image
img = cv2.imread("path/to/your/image.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)

# Draw circles around the detected corners
for corner in corners:
    x, y = corner.ravel()
    cv2.circle(img, (x, y), 5, (255, 0, 0), -1)

# Display the image with corners marked
cv2.imshow("Image with Corners", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

Remember to replace `"path/to/your/image.jpg"` with the actual path to your image file. This code demonstrates how to detect and visualize corners using `goodFeaturesToTrack()`.

By understanding `goodFeaturesToTrack()` and its applications, you can effectively extract valuable features from images for various computer vision tasks in OpenCV.


You're absolutely right! You can indeed use `goodFeaturesToTrack()` in OpenCV for color images. While the function typically works with grayscale images, it can be applied to color images with a minor adjustment.

Here's the key point:

- `goodFeaturesToTrack()` relies on intensity variations to identify corners. These variations are often more pronounced in grayscale images compared to color images, where different color channels might obscure these variations. The correct statement about how `cv2.goodFeaturesToTrack` detects corners is:

- **It looks for areas where the structure tensor has a small eigenvalue**¹³⁴⁵.

`cv2.goodFeaturesToTrack` is a function in OpenCV that finds the N strongest corners in the image by the Shi-Tomasi method (or Harris Corner Detection, if specified)¹³⁴⁵. The function operates on grayscale images and you specify the number of corners you want to find¹³⁴⁵. The Shi-Tomasi method is based on the eigenvalues of the structure tensor of the image¹³⁴⁵. It does not look for areas with high gradient magnitude in the image, apply a threshold to the image and detect local maxima in the resulting binary image, or apply a Laplacian of Gaussian filter to the image and detect zero-crossings¹³⁴⁵. It specifically looks for areas where the structure tensor has a small eigenvalue¹³⁴⁵.

Source: Conversation with Bing, 5/3/2024
(1) Shi-Tomasi corner detector - OpenCV. https://docs.opencv.org/master/d8/dd8/tutorial_good_features_to_track.html.
(2) OpenCV: Shi-Tomasi Corner Detector & Good Features to Track - GitHub Pages. https://gregorkovalcik.github.io/opencv_contrib/tutorial_py_shi_tomasi.html.
(3) Shi-Tomasi Corner Detector & Good Features to Track. http://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_shi_tomasi/py_shi_tomasi.html.
(4) OpenCV: Shi-Tomasi Corner Detector & Good Features to Track. https://docs.opencv.org/3.4/d4/d8c/tutorial_py_shi_tomasi.html.
(5) How to find accurate corner positions of a distorted rectangle from .... https://stackoverflow.com/questions/58736927/how-to-find-accurate-corner-positions-of-a-distorted-rectangle-from-blurry-image.

**Approaches for Using goodFeaturesToTrack() with Color Images:**

1. **Convert to Grayscale:** The most common approach is to convert the color image to grayscale before applying `goodFeaturesToTrack()`. This simplifies the corner detection process by focusing on intensity variations.

2. **Use Individual Color Channels (Experimental):** While less common, you could potentially experiment with using individual color channels (e.g., red, green, or blue) as input to `goodFeaturesToTrack()`. However, the effectiveness of this approach might depend on the specific image content and the dominant color variations that define the corners.

**Recommendation:**

Converting the color image to grayscale is the generally recommended and reliable approach for using `goodFeaturesToTrack()`. This ensures that the corner detection focuses on intensity variations, which are often more robust for identifying stable corners suitable for tracking.

**Here's a revised code example using grayscale conversion:**

```python
import cv2
import numpy as np

# Load an image
img = cv2.imread("path/to/your/image.jpg")

# Convert to grayscale (essential for reliable corner detection)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)

# Draw circles around the detected corners
for corner in corners:
    x, y = corner.ravel()
    cv2.circle(img, (x, y), 5, (255, 0, 0), -1)

# Display the image with corners marked
cv2.imshow("Image with Corners", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

By converting to grayscale before applying `goodFeaturesToTrack()`, you can effectively detect corners in color images as well. Remember to replace `"path/to/your/image.jpg"` with the actual path to your image file.


The following code snippet in the while loop:

```python
cv2.imshow(win_name, result)

key = cv2.waitKey(1)
if key == ord("Q") or key == ord("q") or key == 27:
    alive = False
elif key == ord("C") or key == ord("c"):
    pass
elif key == ord("B") or key == ord("b"):
    pass
elif key == ord("F") or key == ord("f"):
    pass
elif key == ord("P") or key == ord("p"):
    pass
```

This block of code here is simply monitoring the keyboard for user input. The script was written so that the run modes could be toggled interactively. For example, if you were running in preview mode and you wanted to transition to canny detection mode, you would simply type a C on the keyboard.


In [ ]:
source.release()
cv2.destroyWindow(win_name)

The code snippet `source.release()` and `cv2.destroyWindow(win_name)` is used to release the video capture object and destroy the OpenCV window, respectively, after you're done using them.

`source.release()` releases the video capture object, freeing up any resources associated with it, such as the camera device.

`cv2.destroyWindow(win_name)` destroys the OpenCV window with the specified window name (`win_name`), removing it from the screen and freeing up any resources associated with it.

These cleanup steps are important to ensure that resources are properly released and the program exits gracefully.


You would simply type a C on the keyboard. That's all there is, there really isn't very much code required to implement this and at this point we're ready to go ahead and execute the script. We'll cycle through the different run options and talk a little bit about the results that we see. This is the preview mode and here we're simply sending the video stream from the camera to the output window in the display.

What I'd like to do next is toggle through the other filters that we implemented and we'll start with the blurring filter. So I'm going to type a B on the keyboard and you can see that the image has been slightly blurred. There's a few reasons you might want to do this. For example, if you had a noisy image, you could apply a small amount of blurring and still obtain an aesthetically pleasing result. But more importantly, in computer vision and image processing, we often use blurring as a pre-processing step to performing feature extraction.

The reason for that is that most feature extraction algorithms use some kind of numerical gradient computation. Performing numerical gradients on raw pixel data can be a rather noisy and not well-behaved process. So smoothing the image prior to performing gradients turns out to be much more robust and well-behaved. That's one of the primary reasons we use blurring in computer vision.

Now let's take a look at the next option which is the corner feature detector. Now I've turned that mode on and you can see a small amount of corner features in the image. There's some here on the microphone, there's a few around my face here and in particular, there's several there in the painting of the horses behind me.

We're going to talk a little bit about how these features are generated based on the input arguments that we selected. There were two input arguments that we talked about in particular. One was a minimum distance between features which is fairly straightforward and the other one was a quality level of the features.

First, we'll talk about the minimum distance. Here I've got a textbook with some very well-defined characters on the front that have nice sharp edges and we're detecting all kinds of corners in those characters. You can see that each of those letters probably has two maybe three corner features for each character. But when I move the book much closer to the camera, you'll see now that there are several more corner features associated with each character. The reason for that is that those letters are much larger in pixel space so now I'm not constrained by the minimum distance between the features because I've made the letters so much larger in pixel space.

Then one other thing I'd like to talk about is this section of the book here with this graphic image on it. If I put this very close to the camera, we're going to see that we detect all kinds of corners especially with the dots and that pattern of the book there. The reason those are jumping around so much is that I'm having a hard time holding the book really still but the main point is that I'm getting all these detections here.

Now watch what happens when I lower the book and expose the text from the title of the book. As soon as I expose the text from the title, all the features associated with the graphic image below have been filtered out. So let's take a look at that again. I raised the book and I get all these features here and now when I lower the book and expose the title of the book, all those have been filtered out.

The reason for that is that the quality level threshold we talked about is based on the highest score for a corner feature in the entire image. Because the corner features associated with these characters in the title of the book are so much stronger, their feature score is higher and therefore we're effectively raising the detection threshold for corner features. I just thought that was an interesting example of how that parameter is actually used and how it can affect the algorithm in your particular application.

Finally, let's go ahead and cycle to the canny edge detection. So, toggle to that mode and so now you can see the results of edge detection here. You can see the microphone is very well defined, the corner of my shoulder against the light background of the wall is very well defined and then up here, behind my shoulder, you see a painting of some horses and the subject matter in that painting is partially defined but there's a lot of broken edges in that painting.

So I thought it'd be interesting to talk about the threshold inputs for the canny edge detector and see if we can improve what that looks like. So before we do that, I'm going to make a screen snap of this video feed just so we can have something to compare to. So I'll put this aside and now I'm going to edit the threshold for the canny edge detector. So previously the lower threshold was very close to the upper threshold so there wasn't much opportunity for us to find some weaker edges and connect them to stronger edges.

But if I lower this to something like 80, we now have an opportunity to consider weaker edges that might be associated with the stronger edges. So I'm going to go ahead and run this and now we'll put these side by side for a comparison. So if you take a look at the image down below, you can see that again the outline of the horses was rather broken in some places and now if you compare that to the video stream up above, you can see that there's been some improvement. We're effectively extending the definition of these edges because we're allowing those edges to be connected to weaker edges that were in between those two thresholds rather than discarding those edges all together. So I thought that was an interesting way.